In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import keras
import keras.backend as K
from keras.layers import Flatten
from keras.datasets import imdb

from keras.preprocessing import sequence

from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

Using TensorFlow backend.


Cheking the index of a word in dictionary:

In [ ]:
keras.datasets.imdb.get_word_index(path="imdb_word_index.json")['hello']

4822

As you can see, the stop words are also included inside this dataset:

In [ ]:
keras.datasets.imdb.get_word_index(path="imdb_word_index.json")['the']

1

***Note:*** The words have been added to the list by considering the frequencies. For example, the most frequent word in English is "the" which has the first index in vocabulary.

How can we ***remove*** these frequent words? 

By skipping most frequent words using skip_top!

"skip_top: skip the top N most frequently occurring words (which may not be informative). These words will appear as oov_char value in the dataset. Defaults to 0, so no words are skipped.",[[1]](https://keras.io/api/datasets/reuters/)

Here, I supposed that we have one hundred and twenty-seven stop words in this dataset. Therefore, I removed one hundred and twenty-seven frequent words in the vocab and replaced them with "removed".

In [ ]:
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size, skip_top=127, oov_char= 00 )

print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))
print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])

print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))

print('Minimum review length: {}'.format(
len(min((X_train + X_test), key=len))))

Loaded dataset with 25000 training samples, 25000 test samples
---review---
[0, 0, 365, 1234, 0, 1156, 354, 0, 0, 0, 0, 0, 1016, 0, 0, 356, 0, 0, 1349, 500, 746, 0, 200, 0, 4132, 0, 0, 0, 1117, 1831, 0, 0, 4831, 0, 0, 0, 4183, 0, 369, 0, 215, 1345, 143, 0, 0, 1838, 0, 1974, 0, 0, 0, 257, 0, 0, 486, 0, 0, 0, 0, 0, 271, 0, 196, 0, 949, 4121, 0, 0, 0, 0, 2212, 2436, 819, 0, 0, 0, 0, 180, 0, 227, 0, 0, 2494, 0, 0, 423, 0, 168, 0, 0, 0, 0, 0, 665, 0, 270, 0, 0, 0, 197, 0, 161, 0, 0, 0, 0, 0, 0, 419, 665, 0, 0, 0, 0, 0, 0, 2084, 0, 4773, 0, 0, 0, 1901]
---label---
1
Maximum review length: 2697
Minimum review length: 70


In [ ]:
# simple check 
for i in range(len(X_train)):
  for j in range(len(X_train[i])):
    if X_train[i][j]==126:
      print(i, j , '127th stop word did not removed')

In [ ]:
#max_words = 500
#X_train = sequence.pad_sequences(X_train, maxlen=max_words)
#X_test = sequence.pad_sequences(X_test, maxlen=max_words)

X_train = sequence.pad_sequences(X_train)
X_test = sequence.pad_sequences(X_test)
print('shape train:',X_train.shape, 'shape test:', X_test.shape)

shape train: (25000, 2494) shape test: (25000, 2315)


In [ ]:
max_words = 2315
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

print('shape train:',X_train.shape, 'shape test:', X_test.shape)

shape train: (25000, 2315) shape test: (25000, 2315)


In [ ]:
embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=X_train.shape[1]))
model.add(LSTM(70))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2315, 32)          160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 70)                28840     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 71        
Total params: 188,911
Trainable params: 188,911
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
			 
batch_size = 256
num_epochs = 10
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24744 samples, validate on 256 samples
Epoch 1/10
24744/24744 [==============================] - 357s 14ms/step - loss: 0.6590 - accuracy: 0.6365 - val_loss: 0.6510 - val_accuracy: 0.5742
Epoch 2/10
24744/24744 [==============================] - 349s 14ms/step - loss: 0.4651 - accuracy: 0.7728 - val_loss: 0.3259 - val_accuracy: 0.8789
Epoch 3/10
24744/24744 [==============================] - 350s 14ms/step - loss: 0.3064 - accuracy: 0.8724 - val_loss: 0.2721 - val_accuracy: 0.8828
Epoch 4/10
24744/24744 [==============================] - 348s 14ms/step - loss: 0.2615 - accuracy: 0.8963 - val_loss: 0.2791 - val_accuracy: 0.8750
Epoch 5/10
24744/24744 [==============================] - 351s 14ms/step - loss: 0.2860 - accuracy: 0.8868 - val_loss: 0.3313 - val_accuracy: 0.8672
Epoch 6/10
24744/24744 [==============================] - 353s 14ms/step - loss: 0.2381 - accuracy: 0.9080 - val_loss: 0.2451 - val_accuracy: 0.9062
Epoch 7/10
24744/24744 [==============================] - 